## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [141]:
df_ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [149]:
# Создаем новый датафрейм со статистикой количества поставленных оценок для каждого пользователя
# Также добавляем для каждого пользователя минимальное и максимальное время
df_ratings_agg = (df_ratings[['userId', 'rating', 'timestamp']].groupby(by='userId')
                                                               .agg({'rating': 'count', 'timestamp': ['min', 'max']}))
# Заменяем мультииндекс в названиях столбцов
df_ratings_agg.columns = ['rating_count', 'timestamp_min', 'timestamp_max']
# Отфильтровываем значение количества оценок более 100
df_ratings_agg = df_ratings_agg[df_ratings_agg.rating_count > 100]
# Находим время жизни кажого пользователя
df_ratings_agg['lifetime'] = df_ratings_agg['timestamp_max'] - df_ratings_agg['timestamp_min']
df_ratings_agg[['rating_count', 'lifetime']][:10]

,rating_count,lifetime
userId,,
4,204,203560
8,116,85187
15,1700,471393496
17,363,8053
19,423,5282
21,162,1365432
22,220,91491
23,726,18342129
26,172,20267261


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [167]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [170]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [171]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [172]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [176]:
# Объединяем датафреймы rzd, auto и air. Так как во всех датафреймах у нас только один столбец с одинаковым именем 
# (client_id) и именно по нему нужно делать объединение, то можно не указывать параметр on в методе merge.
# Для сохранения всех данных выбираем способ outer
client_revenue = rzd.merge(auto, how='outer').merge(air, how='outer')

print('Данные по клиентам без указания адресов')
display(client_revenue)

Данные по клиентам без указания адресов


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [180]:
# Так как нам нужно отобразить адреса только для тех клиентов, которые у нас уже есть в датафрейме без потери данных,
# то приемняем метод left
client_revenue_with_address = pd.merge(client_revenue, client_base, how='left')

print('Данные по клиентам c указанием адресов')
display(client_revenue_with_address)

Данные по клиентам c указанием адресов


,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Задание вызывает очень много дополнительных вопросов, непонятна структура таблицы визитов и покупок, в каком формате нам отдают геокоординаты, есть ли там еще доп. инфомрация, например, время поулчения этих геокоординат. Поэтому буду придумывать по ходу.

У каждого визита / факта покупки есть определенная дата и время. Соответственно, посетитель мог совершать покупку или посещать магазин несколько раз из разных точек земного шара, следовательно количество записей для одного пользователя в таблице визитво может быть несколько и нам нужно понимать как координаты конкретного пользователя относятся к конкретному визиту этого пользователя. Следовательно, если у геокоординат есть время их получения, мы можем по времени геокоординат и по идентификатору клиента соотнести их с покупкой. Время получения геокоординат должно быть примерно равно времени покупки или входу на сайт с некоторой погрешностью. Если для одной и той же даты находится несколько пар координат, то можно попробовать вычислять область круга, в который входят все эти координаты, находить его центр и хранить координаты центра круга и его радиус в таблице визитов. На основании такого круга мы можем понять из какого города человек совершил покупку, какое время суток было и на сонове этих данных можно проанализировать где наш продукт наиболее популярен и в какое время обычно люди совершают покупки у нас.